# Organize and share dataset versions

<a target="_blank" href="https://colab.research.google.com/github/neptune-ai/examples/blob/main/how-to-guides/data-versioning/notebooks/Organize_and_share_dataset_versions.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/>
</a><a target="_blank" href="https://github.com/neptune-ai/examples/blob/main/how-to-guides/data-versioning/notebooks/Organize_and_share_dataset_versions.ipynb">
  <img alt="Open in GitHub" src="https://img.shields.io/badge/Open_in_GitHub-blue?logo=github&labelColor=black">
</a><a target="_blank" href="https://app.neptune.ai/o/common/org/data-versioning/metadata?path=datasets%2Ftrain_sampled&attribute=latest&filePath=.&file=train_sampled.csv"> 
  <img alt="Explore in Neptune" src="https://neptune.ai/wp-content/uploads/2024/01/neptune-badge.svg">
</a><a target="_blank" href="https://docs.neptune.ai/tutorials/project_metadata/">
  <img alt="View tutorial in docs" src="https://neptune.ai/wp-content/uploads/2024/01/docs-badge-2.svg">
</a>

## Introduction

You can log and query metadata at a project level, including dataset and model versions, text notes, images, notebook files, and anything else you can log to a single run.

This guide shows how to:
* Log versions of all the datasets used in a project
* Organize dataset version metadata in the Neptune app
* Share all the currently used dataset versions with your team
* Assert that you are training on the latest dataset version available

By the end of this guide, you will log various dataset versions, organize them in the Neptune app and see how to share them with a persistent link.

![image](https://neptune.ai/wp-content/uploads/neptune-datasets.png)

## Before you start

This notebook example lets you try out Neptune as an anonymous user, with zero setup.

If you want to see the example logged to your own workspace instead:

  1. Create a Neptune account. [Register &rarr;](https://neptune.ai/register)
  1. Create a Neptune project that you will use for tracking metadata. For instructions, see [Creating a project](https://docs.neptune.ai/setup/creating_project) in the Neptune docs.

## Download data

In [ ]:
! curl https://raw.githubusercontent.com/neptune-ai/examples/main/how-to-guides/data-versioning/datasets/tables/train.csv --create-dirs -o ../datasets/tables/train.csv
! curl https://raw.githubusercontent.com/neptune-ai/examples/main/how-to-guides/data-versioning/datasets/tables/test.csv --create-dirs -o ../datasets/tables/test.csv
! curl https://raw.githubusercontent.com/neptune-ai/examples/main/how-to-guides/data-versioning/datasets/tables/train_v2.csv --create-dirs -o ../datasets/tables/train_v2.csv

## Install Neptune and dependencies

In [ ]:
! pip install -U neptune scikit-learn

## Initialize the Neptune project 

To create a new run for tracking the metadata, you tell Neptune who you are (`api_token`) and where to send the data (`project`).

You can use the default code cell below to create an anonymous run in a public project. **Note**: Public projects are cleaned regularly, so anonymous runs are only stored temporarily.

### Log to your own project instead

Replace the code below with the following:

```python
import neptune
from getpass import getpass

run = neptune.init_run(
    project="workspace-name/project-name",  # replace with your own (see instructions below)
    api_token=getpass("Enter your Neptune API token: "),
)
```

To find your API token and full project name:

1. [Log in to Neptune](https://app.neptune.ai/).
1. In the bottom-left corner, expand your user menu and select **Get your API token**.
1. The workspace name is displayed in the top-left corner of the app. 

    To copy the project path, in the top-right corner, open the settings menu and select **Properties**.

For more help, see [Setting Neptune credentials](https://docs.neptune.ai/setup/setting_credentials) in the Neptune docs.

In [ ]:
import neptune

project = neptune.init_project(
    project="common/data-versioning", api_token=neptune.ANONYMOUS_API_TOKEN
)

**To open the run in the Neptune web app, click the link that appeared in the cell output.**

We'll use the `run` object we just created to log metadata. You'll see the metadata appear in the app.

## Log various dataset versions to Neptune

Create a few different training data samples and log them as different dataset versions to a Neptune project. 

In [ ]:
import pandas as pd

train = pd.read_csv("../datasets/tables/train.csv")

for i in range(5):
    train_sample = train.sample(frac=0.5 + 0.1 * i)
    train_sample.to_csv("../datasets/tables/train_sampled.csv", index=None)
    project[f"datasets/train_sampled/v{i}"].track_files(
        "../datasets/tables/train_sampled.csv", wait=True
    )

---
**Note**

In this case, you need to use ``wait=True`` to ensure all the logging operations are finished. 
By default, Neptune logs almost everything asynchronously.

---

You can confirm that it was logged by looking at the project metadata structure in the **datasets** namespace. 

In [ ]:
project.get_structure()["datasets"]

**You should see something like this:**

{'train_sampled': {'latest': <neptune.attributes.atoms.artifact.Artifact at 0x117af5b40>,
  'v0': <neptune.attributes.atoms.artifact.Artifact at 0x117af6ef0>,
  'v1': <neptune.attributes.atoms.artifact.Artifact at 0x117b491e0>,
  'v2': <neptune.attributes.atoms.artifact.Artifact at 0x117b49240>,
  'v3': <neptune.attributes.atoms.artifact.Artifact at 0x117b492a0>,
  'v4': <neptune.attributes.atoms.artifact.Artifact at 0x117b49300>}}

Get the latest version of a dataset and create a separate dataset version called 'latest'.

In [ ]:
def get_latest_version():
    artifact_name = project.get_structure()["datasets"]["train_sampled"].keys()
    versions = [int(version.replace("v", "")) for version in artifact_name if version != "latest"]
    return max(versions)


latest_version = get_latest_version()
print("latest version", latest_version)

In [ ]:
project["datasets/train_sampled/latest"].assign(
    project[f"datasets/train_sampled/v{latest_version}"].fetch(), wait=True
)

## See dataset versions in the Neptune app and share them with the team

You can get a list of all datasets used in a project by running ``project.get_structure()`` function.

In [ ]:
project.get_structure()["datasets"]

**You should see something like this:**

{'train_sampled': {'latest': <neptune.attributes.atoms.artifact.Artifact at 0x117af5b40>,
  'v0': <neptune.attributes.atoms.artifact.Artifact at 0x117af6ef0>,
  'v1': <neptune.attributes.atoms.artifact.Artifact at 0x117b491e0>,
  'v2': <neptune.attributes.atoms.artifact.Artifact at 0x117b49240>,
  'v3': <neptune.attributes.atoms.artifact.Artifact at 0x117b492a0>,
  'v4': <neptune.attributes.atoms.artifact.Artifact at 0x117b49300>}}

You and your team can also see and access all that in the Neptune app.
Go to your project and then **Project Metadata > datasets**

![image](https://neptune.ai/wp-content/uploads/Screenshot-from-2021-12-23-14-00-51.png)

As all links in the Neptune app, the URL of the Project metadata for your project is persistent. 

For example:

https://app.neptune.ai/o/common/org/data-versioning/metadata?path=datasets%2Ftrain_sampled&attribute=latest

## Create a new Neptune run

Connect your script to the Neptune application and create a new run.

In [ ]:
run = neptune.init_run(project="common/data-versioning", api_token=neptune.ANONYMOUS_API_TOKEN)

## Assert that you are training on the latest dataset version

Log dataset version of the dataset you want to train your models on as a Neptune artifact

In [ ]:
TRAIN_DATASET_PATH = "../datasets/tables/train_sampled.csv"
run["datasets/train"].track_files(TRAIN_DATASET_PATH, wait=True)

Assert that it is the same dataset as the latest dataset version in your project. 

In [ ]:
assert run["datasets/train"].fetch_hash() == project["datasets/train_sampled/latest"].fetch_hash()

**Note:**

You can also download the latest version of the dataset by running 

```python
project["datasets/train_sampled/latest"].download()
```

## Run model training and log parameters and metrics to Neptune

Now train a model and log the test score to Neptune.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

TEST_DATASET_PATH = "../datasets/tables/test.csv"

params = {
    "n_estimators": 8,
    "max_depth": 3,
    "max_features": 2,
}
run["parameters"] = params

train = pd.read_csv(TRAIN_DATASET_PATH)
test = pd.read_csv(TEST_DATASET_PATH)

FEATURE_COLUMNS = ["sepal.length", "sepal.width", "petal.length", "petal.width"]
TARGET_COLUMN = ["variety"]
X_train, y_train = train[FEATURE_COLUMNS], train[TARGET_COLUMN]
X_test, y_test = test[FEATURE_COLUMNS], test[TARGET_COLUMN]

rf = RandomForestClassifier(**params)
rf.fit(X_train, y_train)

score = rf.score(X_test, y_test)
run["metrics/test_score"] = score

## Stop logging

Once you are done logging, stop tracking the run.

In [ ]:
run.stop()
project.stop()